In [6]:
import numpy as np
import pandas as pd
import os
import sys
import time
import datetime
import pickle
from collections import defaultdict, namedtuple, deque
import random
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F


In [2]:
df_baggage = pd.read_csv("./data/box_size.csv")
df_baggage.sample(1)

,w,h,d
1969,540,190,190


In [3]:
D = 600
H = 680
W = 1120

# temporary rotation
def rotate_action(r: int, HWD: list):
  if r == 0:
    return
  elif r == 1:
    HWD[0], HWD[1] = HWD[1], HWD[0]
  elif r == 2:
    HWD[0], HWD[2] = HWD[2], HWD[0]
  elif r == 3:
    HWD[1], HWD[2] = HWD[2], HWD[1]
  elif r == 4:
    HWD[0], HWD[1], HWD[2] = HWD[1], HWD[2], HWD[0]
  elif r == 5:
    HWD[0], HWD[1], HWD[2] = HWD[2], HWD[0], HWD[1]
  else:
    print("Invalid Operation")
    sys.exit(1)

def sample_baggage():
  while True:
    size_list = df_baggage.sample(1).to_list()
    r = np.random.randint(0, 6)
    rotate_action(r=r, size_list=size_list)
    if size_list[0] > H or size_list[1] > W or size_list[2] > D:
      continue
    else:
      return size_list

def calc_cross_area():



In [ ]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward', 'done'))

class ReplayBuffer(object):
    def __init__(self, capacity=1e6):
        self.capacity = capacity
        self.memory = deque([], maxlen=int(capacity))

    def append(self, *args):
        transition = Transition(*args)
        self.memory.append(transition)

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def reset(self):
        self.memory.clear()

    def length(self):
        return len(self.memory)

    def __len__(self):
        return len(self.memory)


class Actor(nn.Module):
    def __init__(self, num_state, num_action, hidden1_size=400, hidden2_size=300, init_w=3e-3):
        super(Actor, self).__init__()
        self.conv1 = nn.Conv2d(kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(kernel_size=5, padding=2)
        self.conv4 = nn.Conv2d(kernel_size=5, padding=2)
        self.conv5 = nn.Conv2d(kernel_size=5, padding=2)

        self.num_state = num_state
        self.num_action = num_action


    def forward(self, x, mask):
        x = F.relu(F.batch_norm(self.conv1(x)))
        x = F.relu(F.batch_norm(self.conv2(x)))
        x = F.relu(F.batch_norm(self.conv3(x)))
        x = F.relu(F.batch_norm(self.conv4(x)))
        x = F.relu(F.batch_norm(self.conv5(x)))
        x = x.masked_fill(mask, -1e18).view(-1).softmax()
        idx = torch.argmax(x)
        pos = (idx//H, idx%W)
        return pos

class Critic(nn.Module):
    def __init__(self, num_state, num_action, hidden1_size=400, hidden2_size=300, init_w=3e-4):
        super(Critic, self).__init__()
        self.conv1 = nn.Conv2d(kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(kernel_size=5, padding=2)
        self.conv4 = nn.Conv2d(kernel_size=5, padding=2)
        self.conv5 = nn.Conv2d(kernel_size=5, padding=2)
        
        self.num_state = num_state
        self.num_action = num_action


    def forward(self, x, action):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(torch.cat([h, action], dim=1)))
        y = self.fc3(h)
        return y
